In [1]:
import os, sys

os.chdir('D:\Downloads\@Books\@TPM\@PDPC\@data_analysis\model_build')
sys.path.append('D:\Downloads\@Books\@TPM\@PDPC\@data_analysis\model_build')
# ^ just a sanity thing: console starts from project root, but pycharm 'run in console' runs with 'model_build' as working directory

import OFM_model as ofm
import pandas as pd
import numpy as np
# import itertools
# import seaborn as sns
# import matplotlib.pyplot as plt
# from matplotlib.colors import TwoSlopeNorm
# import cmasher as cmr
# import colorcet as cet
from importlib import reload  ## for reloading packages
import scipy
# import pickle

In [2]:
# reload(ofm)
# test_i_flights = pd.Series(np.arange(0, 20))
# test_i_nonflights = pd.Series([1]*20)
# test_i_other = pd.Series({5:1})
# test_i_blank = pd.Series(dtype='float64')
#
# test_model = ofm.ISIRmodel_SingleRun(
#     R_zero=1,
#     t_incubation=1,
#     func_infectious=[0,0,0],
#     pop_total=100,
#     sim_time=100,
#     imports_flights=test_i_flights,
#     imports_indirect=test_i_nonflights,
#     imports_other=test_i_other,
#     flightban_on=10,
# )

In [3]:
imports_path = "data_input/omi_prevalence_flights_not_inflated.pickletable"

imports_df = pd.read_pickle(imports_path)
p_flightbans = range(-7, 57, 1)  # 1-day increments since -7 (week earlier) till 56 days
s_flightbans = range(-3, 4, 1)
R_range = list(np.arange(11, 31, 1) / 10)

model_vers = 'v2'
pop_NL = int(17.48e6)  # 17.48 million




In [4]:
# generate infectious distribution for v2
available_distributions = {
    'alpha': (2.0, 0.44),
    'delta': (2.11, 0.50),
    'omicron': (2.14, 0.59)
}

a = 2.14
rate = .59
x = range(0, 20, 1)
func_serial = scipy.stats.gamma.pdf(x=x, a=a, scale=1 / rate, loc=-1)
func_serial_cdf = scipy.stats.gamma.cdf(x=x, a=a, scale=1 / rate, loc=-1)  # for illustration
# design a cutoff
# Note: truncation of day 0 entry


In [5]:
test_i_nonflights = pd.Series([1]*20)
test_i_other = pd.Series({5:100})
test_i_blank = pd.Series(dtype='float64')

In [62]:
reload(ofm)
experiments = ofm.ISIR_PolicyExperiments(
 u_ImportsFunc=imports_df['import_pax'],
 p_FlightBans=tuple(s_flightbans) + (None,),
 u_ImportsIndirect=None,
 u_ImportsOther=None, #(pd.Series({10:10}), pd.Series({20:20})),
 u_Rzero=(1.5,),
 u_ImportsFlights=(15,),
 u_Func_Infectious= None, #(func_serial,),
 c_SimTime=300,
 c_import_scaling_mode=2,  # **
 c_model_engine='step_v1_constant_beta',
 # debug_mode=True
                                         )

OFM_PE Warning: OFM_model has non-integer flightban value(s) [None]. If this is intentional, do proceed.


In [63]:
experiments.run_experiments()
# debuggo = experiments.debug_experiment

OFM_PE: Running 8 experiments
OFM_PE: all experiments completed in 0.0 mins
